In [72]:
import os
import numpy as np
import random
import csv
import matplotlib.pyplot as plt
from dataset import dataset
from additional_functions import process_all

In [73]:
class neural_net:
    def __init__(self, data: dataset, prediction_type_flag: str, hidden_layer_count=0, momentum=1.0, learning_rate=1.0, batch_size=1, hidden_node_count=1, suppress_plots=True):
        self.suppress_plots = suppress_plots
        self.hidden_layer_count = hidden_layer_count
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        if hidden_layer_count == 0:
            self.hidden_node_count = 0
        else:
            self.hidden_node_count = hidden_node_count
        self.tune_set = data.tune_set
        self.validate_set = data.validate_set
        self.prediction_type = prediction_type_flag
        self.predictions = []
        self.answers = []
    def init_weights(self, num_hidden_nodes: int):
        '''
        This function currently returns a list of 
        '''
        input_size = self.tune_set.shape[1] - 1
        hidden_size = num_hidden_nodes
        output_size = 1 if self.prediction_type == "regression" else input_size
        seam_count = self.hidden_layer_count + 1
        all_weights = []
        weights_per_seam = []
        weight_count = 0
        for seam_idx in range(seam_count):
            # enters on first iteration
            if seam_idx == 0:
                if hidden_size == 0:
                    weight_count = (input_size * output_size)
                else:
                    weight_count = (input_size * hidden_size)
            # enters on the last iteration
            elif seam_idx == (seam_count - 1):
                weight_count = (hidden_size * output_size)
            # enters on iterations aside from first or last
            else:
                weight_count = (hidden_size * hidden_size)
            weights_per_seam = np.random.uniform(low=-.5, high=.5, size=weight_count)
            # THIS CURRENTLY MAKES A LIST FULL OF NUMPY ARRAYS THAT WILL HAVE DIFFERENT DIMENSIONS,
            # SO all_weights CANNOT BE CONVERTED TO A NUMPY ARRAY. CONSIDER ADDING PADDING TO MAKE all_weights NUMPY
            all_weights.append(weights_per_seam)
        # PAD all_weights HERE
        return all_weights

    def for_prop(self, example_set: np):
        return
    '''
    # consider using these instead of for_prop, or make a class for each layer type
    def input_layer(self):
        return
    def hidden_layer(self):
        return
    def output_layer(self):
        return
    '''
    def grad_desc(self):
        return
    def mini_batch(self):
        return
    def tune(self):
        return
    def train(self):
        return
    def loss(self):
        return
    def loss_prime(self):
        return
    '''
    # Consider using this to dynamically choose the output activation function
    def activation(self, prediction_type: str):
        return
    '''
    def sigmoid(self):
        return
    def sigmoid_prime(self):
        return
    def softmax(self):
        return
    '''
    # Since loss output is in a slightly different format for neural nets, we might need an evaluate method to output final performance
    def evaluate(self):
        return
    '''
    # THIS PROLLY NEEDS EDITING
    def plot_loss(self, metrics: list, parameter: str, increment):
        '''
        This function plots the loss performance for each epoch. This allows us to visualize at how many epochs
        performance drops off.
        '''
        # Extract # of epochs and loss metrics
        metrics = np.array(metrics)
        epochs = np.arange(1, metrics.shape[0] + 1) * increment
        loss1 = metrics[:, 0]
        loss2 = metrics[:, 1]

        # Create loss plot
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, loss1, label='Loss Metric 1', marker='o')
        plt.plot(epochs, loss2, label='Loss Metric 2', marker='o')
        plt.xlabel(f'{parameter} Value')
        plt.ylabel('Loss')
        plt.title(f'Loss Metrics vs. {parameter} value')
        plt.legend()
        plt.grid(True)
        plt.show()
        plt.close()
    

In [74]:
abalone_data, cancer_data, fire_data, glass_data, machine_data, soybean_data = process_all('carlthedog3', True)

In [75]:
num_hidden_layers = 2
cancer_net = neural_net(cancer_data, "classification", hidden_layer_count=num_hidden_layers, hidden_node_count=3)
x = cancer_net.init_weights(5)
for i in range(num_hidden_layers+1):
    print(x[i].shape)
print("\n")
num_hidden_layers = 0
fire_net = neural_net(fire_data, "regression", hidden_layer_count=num_hidden_layers, hidden_node_count=3)
x = fire_net.init_weights(0)
for i in range(num_hidden_layers+1):
    print(x[i].shape)

(45,)
(25,)
(45,)


(12,)
